# Extended ra operations
> relation algebra operations and ie function calculation operations

In [ ]:
#| default_exp ra

In [ ]:
"""
TODO how to solve calc outputs binding to different inputs

 we cant simply join inputs and outputs, because then outputs from different inputs can intermingle

 # we need to keep track of the input index and join inputs and their outputs on free vars plus input index
 # maybe we want to also inline the renaming into the calc node

"""

'\nTODO how to solve calc outputs binding to different inputs\n\n we cant simply join inputs and outputs, because then outputs from different inputs can intermingle\n\n # we need to keep track of the input index and join inputs and their outputs on free vars plus input index\n # maybe we want to also inline the renaming into the calc node\n\n'

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
#| export
import pytest
import pandas as pd
import numpy as np
from typing import no_type_check, Set, Sequence, Any,Optional,List,Callable,Dict,Union
import networkx as nx
import itertools

from spannerlib.utils import serialize_df_values
from spannerlib.span import Span
from spannerlib.data_types import _infer_relation_schema,pretty

import logging
logger = logging.getLogger(__name__)

In [ ]:
#| export
def _col_names(length):
    # these names wont conflixt with logical variables since they must always start with Uppercase letters
    return [f'col_{i}' for i in range(length)]



In [ ]:
s = pd.DataFrame([
    [1,1],
    [2,2],
    [3,3],
    [4,5]
])

s2 = pd.DataFrame([
    [1,2,3],
    [2,3,4],
    [2,3,5],
    [4,5,6]
])

In [ ]:
df = pd.DataFrame([
    [1,2],
    [1,3],
    [1,2],
    ])
df

,0,1
0,1,2
1,1,3
2,1,2


### RA

In [ ]:
#| export

# some select theta functions

class equalConstTheta():
    def __init__(self,*pos_val_tuples):
        self.pos_val_tuples = pos_val_tuples
    def __call__(self,df):
        masks = [df.iloc[:,pos]==val for pos,val in self.pos_val_tuples]
        return pd.concat(masks,axis=1).all(axis=1)
    def __str__(self):
        return f'''Theta({', '.join([f'col_{pos}={val}' for pos,val in self.pos_val_tuples])})'''
    def __repr__(self):
        return str(self)
    def __eq__(self,other):
        if not isinstance(other,equalConstTheta):
            return False
        return self.pos_val_tuples == other.pos_val_tuples

class equalColTheta():
    def __init__(self,*col_pos_tuples):
        self.col_pos_tuples = col_pos_tuples

    def __call__(self,df):
        masks = [df.iloc[:,pos1]==df.iloc[:,pos2] for pos1,pos2 in self.col_pos_tuples]
        return pd.concat(masks,axis=1).all(axis=1)    
    def __str__(self):
        return f'''Theta({', '.join([f'col_{pos1}=col_{pos2}' for pos1,pos2 in self.col_pos_tuples])})'''
    def __repr__(self):
        return str(self)
    def __eq__(self,other):
        if not isinstance(other,equalColTheta):
            return False
        return self.col_pos_tuples == other.col_pos_tuples

In [ ]:
s4 = pd.DataFrame([
    [1,2,3,1],
    [1,4,4,1],
    [1,2,3,1],
    [1,4,4,0]
])
s4

,0,1,2,3
0,1,2,3,1
1,1,4,4,1
2,1,2,3,1
3,1,4,4,0


In [ ]:
s4.iloc[:,0]==1

0    True
1    True
2    True
3    True
Name: 0, dtype: bool

In [ ]:
assert list(equalConstTheta((0,1),(2,4))(s4)) == [False, True, False, True]
assert list(equalColTheta((0,3),(1,2))(s4)) == [False, True, False, False]

 

In [ ]:
#| export
def get_const(const_dict,**kwargs):
    return pd.DataFrame([const_dict])

In [ ]:
res = get_const({'_C1':1,'_C2':2,0:3})
assert serialize_df_values(res) == {(1,2,3)}
assert list(res.columns) == ['_C1','_C2',0]
res

,_C1,_C2,0
0,1,2,3


In [ ]:
#| export
def select(df,theta,**kwargs):
    if df.empty:
        return df

    if callable(theta):
        return df[theta(df)]
    else:
        raise ValueError(f"theta must be callable, got {theta}")

def project(df,on=None,not_on=None,**kwargs):
    if df.empty and len(df.columns)==0:
        return df
    if on is None and not_on is None:
        raise Value("either on or not_on must be specified")
    if on is not None:
        return df[on]
    else:
        return df.drop(columns=not_on)

def rename(df,names,**kwargs):
    if df.empty and len(df.columns)==0:
        return df
    names_mapper = {
    }
    current_col_names = list(df.columns)
    for i,name in names:
        names_mapper[current_col_names[i]] = name
    return df.rename(names_mapper,axis=1)

def union(*dfs,**kwargs):
    # use numpy arrays to ignore column names
    non_empty_dfs = [df for df in dfs if not df.empty]
    if len(non_empty_dfs)==0:
        return dfs[0]
    return pd.DataFrame(np.concatenate(non_empty_dfs,axis=0)).drop_duplicates()

def intersection(df1,df2,**kwargs):
    return pd.merge(df1,df2,how='inner',on=list(df1.columns))

def difference(df1,df2,**kwargs):
    return pd.concat([df1,df2]).drop_duplicates(keep=False)

def join(df1,df2,**kwargs):
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    on = cols1 & cols2
    # get only logical variables
    on = [ col for col in on if isinstance(col,str) and col[0].isupper()]
    if len(on)==0:
        return pd.merge(df1,df2,how='cross')
    else:
        return pd.merge(df1,df2,how='inner',on=on)

def product(df1,df2,**kwargs):
    return pd.merge(df1,df2,how='cross')

#### Tests

In [ ]:
s3 = pd.DataFrame([
    [4,5,6],
    [5,6,7],
    [1,2,3],
    [7,8,9]
])
s3

,0,1,2
0,4,5,6
1,5,6,7
2,1,2,3
3,7,8,9


In [ ]:
s

,0,1
0,1,1
1,2,2
2,3,3
3,4,5


In [ ]:
s2_copy = s2.copy()
s2

,0,1,2
0,1,2,3
1,2,3,4
2,2,3,5
3,4,5,6


In [ ]:
empty = pd.DataFrame()

truth = pd.DataFrame([()])

In [ ]:
empty.empty

True

In [ ]:
pd.DataFrame(columns=['a','b']).empty

True

In [ ]:
assert serialize_df_values(select(empty,None))==set()
assert serialize_df_values(rename(empty,names=[(0,'X')]))==set()
assert serialize_df_values(project(empty,on=['X','Y']))==set()

In [ ]:
s

,0,1
0,1,1
1,2,2
2,3,3
3,4,5


In [ ]:
res = select(s,equalConstTheta((0,1)))
assert serialize_df_values(res)=={(1,1)}

res = select(s2,equalConstTheta((0,2),(1,3)))
assert serialize_df_values(res) == {(2,3,4),(2,3,5)}

res = select(s,equalColTheta((0,1)))
assert serialize_df_values(res) == {(1, 1), (2, 2), (3, 3)}

In [ ]:

s2

,0,1,2
0,1,2,3
1,2,3,4
2,2,3,5
3,4,5,6


In [ ]:
res = project(s2,on=[2,1])
assert serialize_df_values(res) == {(3, 2), (4, 3), (5, 3), (6, 5)}
res = project(s2,not_on=[0,1])
assert serialize_df_values(res)== {(3,), (4,), (5,), (6,)}

In [ ]:
assert list(rename(s2,[(0,'X')]).columns) == ['X',1,2]
assert list(rename(s2,[(0,'X'),(2,'Z')]).columns) == ['X',1,'Z']

In [ ]:
res = union(s2,s3)
assert serialize_df_values(res) == {(1, 2, 3), (2, 3, 4), (2, 3, 5), (4, 5, 6), (5, 6, 7), (7, 8, 9)}

res = intersection(s2,s3) 
assert serialize_df_values(res) == {(1, 2, 3), (4, 5, 6),}

res = difference(s2,s3)
assert serialize_df_values(res) == { (2, 3, 4), (2, 3, 5),(5, 6, 7), (7, 8, 9)}


In [ ]:
d1 =pd.DataFrame([[1,2,1]],columns=['X','Y','Z'])
d2 = pd.DataFrame([[3,4,5]],columns=['X',1,'Y'])
assert serialize_df_values(union(d1,d2)) == {(1, 2, 1), (3, 4, 5)}


In [ ]:
left = rename(s,[(1,'Y')])
left

,0,Y
0,1,1
1,2,2
2,3,3
3,4,5


In [ ]:
right = rename(s2,[(0,'Y'),(1,'X')])
right

,Y,X,2
0,1,2,3
1,2,3,4
2,2,3,5
3,4,5,6


In [ ]:
res = join(left,right)
assert serialize_df_values(res) == {(1, 1, 2, 3), (2, 2, 3, 4), (2, 2, 3, 5)}
res

,0,Y,X,2
0,1,1,2,3
1,2,2,3,4
2,2,2,3,5


In [ ]:
res = join(rename(s,[(0,'a'),(1,'b')]),s)
assert len(res)==16
assert list(res.columns) == ['a', 'b', 0, 1]
res.head()

,a,b,0,1
0,1,1,1,1
1,1,1,2,2
2,1,1,3,3
3,1,1,4,5
4,2,2,1,1


### Calc ie operators

To calculate ie functions we need
* a map like operator to run the function on the input tuples to generate `[input+output]` tuples


In [ ]:
#Each schema is either a const 

# in = [str,int,str]



# in = lambda x: [str] * x
# out = lambda y: y==
# format("{0} {1} {2}",X,Y,Z)->(S)



In [ ]:
#| export
def assert_tuple_like(name,func,input,output):
    if not isinstance(output,(tuple,list)):
        raise ValueError(f"IEFunction {name} with underlying function {func}\n"
                         f"returned a value that is not a tuple/list\n"
                         f"for input output pair ({input},{output})"
                         f"did you remember to return the output as a tuple/list?")

def assert_ie_schema(name,func,value,expected_schema,arity,input_or_output='input'):
    if callable(expected_schema):
        expected_schema = expected_schema(arity)
    actual_schema = [type(v) for v in value]
    if actual_schema != expected_schema:
        raise ValueError(
            f"IEFunction {name} with underlying function {func}\n"
            f"received an {input_or_output} value {value}(schema={pretty(_infer_relation_schema(value))})\n"
            f"but expected {pretty(expected_schema)}")

def assert_iterable(name,func,input,output):
    try:
        out_iter = iter(output)
    except TypeError:
        raise ValueError(f"IEFunction {name} with underlying function {func}\n"
                f"returned a value that is not an iterable\n"
                f"for input {input} -> {output}")

def map_iter(df,name,func,in_schema,out_schema,in_arity,out_arity,**kwargs):
    """helper function returns an iterator that applies a function to each row of a dataframe
    """
    for _,in_row in df.iterrows():
        in_row = list(in_row)
        assert_ie_schema(name,func,in_row,in_schema,in_arity,input_or_output='input')
        output = func(*in_row)
        assert_iterable(name,func,in_row,output)
        for out_row in output:
            assert_tuple_like(name,func,in_row,out_row)
            out_row = list(out_row)
            assert_ie_schema(name,func,out_row,out_schema,out_arity,input_or_output='output')
            yield in_row + out_row

def ie_map(df,name,func,in_schema,out_schema,in_arity,out_arity,**kwargs):
    """given an indexed dataframe, apply an ie function to each row and return the output 
    such that each output relation is indexed by the same index as the input relation that generated it
    """
    # if df.empty:
    #     return df
    output_iter = map_iter(df,name,func,in_schema,out_schema,in_arity,out_arity)
    total_arity = in_arity + out_arity
    return pd.DataFrame(output_iter,columns=_col_names(total_arity))





#### Tests

In [ ]:
s2

,0,1,2
0,1,2,3
1,2,3,4
2,2,3,5
3,4,5,6


In [ ]:
def func(x,y): return [(x+y,x-y)]
def func2(x,y,z): return [(x,y)]

assert serialize_df_values(ie_map(s,'F',func,[int,int],[int,int],in_arity=2,out_arity=2)) == {
    (1, 1, 2, 0), (2, 2, 4, 0), (3, 3, 6, 0), (4, 5, 9, -1)}
assert serialize_df_values(ie_map(s2,'F',func2,
    in_schema=[int,int,int],out_schema=[int,int],
    in_arity=3,out_arity=2)) == {
    (1, 2, 3, 1, 2), (2, 3, 4, 2, 3), (2, 3, 5, 2, 3), (4, 5, 6, 4, 5)
    }

[1, 1]
[2, 2]
[3, 3]
[4, 5]
[1, 2, 3]
[2, 3, 4]
[2, 3, 5]
[4, 5, 6]


In [ ]:
s2_empty = pd.DataFrame(columns=[0,1,2])
ie_map(s2_empty,'F',func2,[int,int,int],[int,int,int],3,3)


,col_0,col_1,col_2,col_3,col_4,col_5


In [ ]:
# test checking of schema
with pytest.raises(ValueError) as exc_info:
    ie_map(s2,'F',func2,[int,int,int],[int,int,int],3,3)
assert 'but expected' in str(exc_info.value)
print(exc_info.value)

with pytest.raises(ValueError) as exc_info:
    ie_map(s2,'F',func2,[int,int,int],[int,str],3,2)
assert 'but expected' in str(exc_info.value)
print(exc_info.value)

# ie function that forgot to return a tuple
not_iter_func = lambda x,y:x+y
not_tuple_iter_func = lambda x,y:[x+y]

with pytest.raises(ValueError) as exc_info:
    ie_map(s,'F',not_iter_func,[int,int],[int],2,1)
assert 'that is not an iterable' in str(exc_info.value)
print(exc_info.value)

with pytest.raises(ValueError) as exc_info:
    ie_map(s,'F',not_tuple_iter_func,[int,int],[int],2,1)
assert 'value that is not a tuple/list' in str(exc_info.value)
print(exc_info.value)

[1, 2, 3]
IEFunction F with underlying function <function func2>
received an output value [1, 2](schema=[<class 'int'>, <class 'int'>])
but expected [<class 'int'>, <class 'int'>, <class 'int'>]
[1, 2, 3]
IEFunction F with underlying function <function func2>
received an output value [1, 2](schema=[<class 'int'>, <class 'int'>])
but expected [<class 'int'>, <class 'str'>]
[1, 1]
IEFunction F with underlying function <function <lambda>>
returned a value that is not an iterable
for input [1, 1] -> 2
[1, 1]
IEFunction F with underlying function <function <lambda>>
returned a value that is not a tuple/list
for input output pair ([1, 1],2)did you remember to return the output as a tuple/list?


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     